<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/Tensorflow_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

!pip install transformers

In [110]:
import tensorflow as tf
from transformers import AutoTokenizer, GPT2Config
from transformers import TFGPT2LMHeadModel

모델 호출

In [113]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

# Add special tokens, including [SEP]
special_tokens = {'bos_token': '</s>', 'eos_token': '</s>', 'pad_token': '<pad>', 'sep_token': '<sep>'}
tokenizer.add_special_tokens(special_tokens)

# Load the existing model checkpoint
model_config = GPT2Config.from_pretrained('skt/kogpt2-base-v2')

# Add the new special token to the model's config

model_config.sep_token_id = tokenizer.sep_token_id  # Add this line


# Initialize the model
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', config=model_config, from_pt = True)

model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'lm_head.weight', 'transformer.h.11.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expe

In [97]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))


1
1
3
----------
</s>
<usr>
<pad>
<sys>


데이터셋 구축

In [6]:
import pandas as pd
import tqdm

In [16]:
# Initialize empty lists to store data
sentence1_list = []
sentence3_list = []
output_list = []

# Read the JSONL file line by line and extract data
with open("/content/drive/MyDrive/nikluge-sc-2023-train.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        data = eval(line)  # Evaluate the JSONL line as a Python dictionary
        sentence1 = data['input']['sentence1']
        sentence3 = data['input']['sentence3']
        output = data['output']
        sentence1_list.append(sentence1)
        sentence3_list.append(sentence3)
        output_list.append(output)

# Create a Pandas DataFrame
df = pd.DataFrame({'sentence1': sentence1_list, 'sentence3': sentence3_list, 'output': output_list})

                                    sentence1  \
0          시은이는 다음 주의 여름 휴가 이전에 기분을 전환하고 싶었다.   
1                   우리는 의자를 큰 책상 주위에 빙 둘러놓았다.   
2               우진이는 폭우로 독서실이 침수되었다는 소식을 들었다.   
3                          나는 줄을 서서 배식을 기다렸다.   
4                  그는 손빨래해야 하는 옷을 건조기에 넣어버렸다.   
...                                       ...   
120135             남자친구와 3주년을 맞이해 여행을 가고 싶었다.   
120136  할머니가 오늘 길에서 봤던 꽃 이름이 기억나지 않는다고 말씀하셨다.   
120137                    아이는 책상에 앉아 있지를 못했다.   
120138              나는 친구들과 음식점에서 큰 소리로 떠들었다.   
120139               가을이 되자 재희는 날씨가 추워짐을 느꼈다.   

                                sentence3  \
0       예쁘게 꾸민 손톱을 보며 여행을 갈 생각에 한층 더 들떴다.   
1                학생들이 모두 착석한 뒤 회의가 시작되었다.   
2       다행히 우진이가 다니는 독서실은 무사해서 우진이는 안도했다.   
3             나는 밥을 국에 말아 먹고, 반찬은 모두 버렸다.   
4               그래서 결국 그는 다시 옷을 구매해야만 했다.   
...                                   ...   
120135             마음 편히 여행 계획을 세울 수 있었다.   
120136    할머니는 아주 노랗고 큰 꽃잎이 네 장이었다고 대답했다.   
120137

In [18]:
train_data = df

In [115]:
batch_size = 16

In [116]:
def get_story_data():
  for sentence1, sentence3, output in zip(train_data.sentence1.to_list(), train_data.sentence3.to_list(), train_data.output.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + sentence1 +'<sep>' + sentence3 + '<sys>' + output)
    yield bos_token + sent + eos_token


In [117]:
train_dataset = tf.data.Dataset.from_generator(get_story_data, output_types=tf.int32)

In [118]:
train_dataset = train_dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [119]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
steps = len(train_data) // batch_size + 1
print(steps)

7509


In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(train_dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)

      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

<ipython-input-121-63f45aad8519>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(train_dataset, total=steps):


  0%|          | 0/7509 [00:00<?, ?it/s]